In [6]:
import cv2
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio
from osgeo import gdal, ogr, osr
from rasterio.crs import CRS
from rasterio.features import rasterize
from rasterio.warp import calculate_default_transform
from shapely.geometry import Point

In [7]:
# 1. 文件路径
input_raster = r"D:\UAV_DATA_NEW\output\2_dilated\061301_dilated.tif"
input_vector = r"D:\UAV_DATA_NEW\output\3_polygonized\061301_polygonized.shp"
filtered_thin_vector = r"D:\UAV_DATA_NEW\output\4_thin\061301_thin.shp"
filtered_fat_vector = r'D:\UAV_DATA_NEW\output\4_fat\061301_fat.shp'
output_raster_thin = r'D:\UAV_DATA_NEW\output\4_thin\061301_thin.tif'
output_raster_fat = r'D:\UAV_DATA_NEW\output\4_fat\061301_fat.tif'

In [8]:
# 提取X、Y方向的像素大小，单个像素面积
src = rasterio.open(input_raster)

transform = src.transform
pixel_width = transform[0]
pixel_height = transform[4]
area = abs(pixel_width * pixel_height)

profile = src.profile
width = src.width
height = src.height

src.close()

interval = 0.62

In [9]:
# 3. 矢量筛选，只保留短边合适的多边形用于统计面积


def cut_polygon(geom):
    """
    Cut OBB rectangle into grids
    """
    obb = geom.minimum_rotated_rectangle.exterior.coords
    p1, p2, p3 = Point(obb[0]), Point(obb[1]), Point(obb[2])
    dist1, dist2 = p1.distance(p2), p2.distance(p3)
    max_dist = max(dist1, dist2)
    return int(round(max_dist / interval) + int(max_dist * 2 < interval))


def _polygon_filter(geom, i) -> bool:
    if cut_polygon(geom.geometry) <= 3:
        return False

    coords = geom.geometry.minimum_rotated_rectangle.exterior.coords
    p1 = Point(coords[0])
    p2 = Point(coords[1])
    p3 = Point(coords[2])
    edge1 = p1.distance(p2)
    edge2 = p2.distance(p3)
    min_edge = min(edge1, edge2)
    return min_edge < 1.15


In [10]:
gdf = gpd.read_file(input_vector)

# filter = gdf.apply(polygon_filter, axis=1)
# thin_gdf = gdf.loc[filter]
# fat_gdf = gdf.loc[~filter]

# thin_gdf.to_file(filtered_thin_vector)
# fat_gdf.to_file(filtered_fat_vector)


In [11]:
# # 胖连通域，需要分割
# out = np.zeros((height, width), dtype=np.uint8)
# rasterize(fat_gdf['geometry'], out=out, transform=transform, default_value=1)

# with rasterio.open(output_raster_fat, 'w', **profile) as dst:
#     dst.write(out, 1)

# # 瘦连通域，不需要分割
# out = np.zeros((height, width), dtype=np.uint8)
# rasterize(thin_gdf['geometry'], out=out, transform=transform, default_value=1)

# with rasterio.open(output_raster_thin, 'w', **profile) as dst:
#     dst.write(out, 1)

In [18]:
cut_num = 2


def polygon_filter(geom) -> bool:
    coords = geom.geometry.minimum_rotated_rectangle.exterior.coords
    p1 = Point(coords[0])
    p2 = Point(coords[1])
    p3 = Point(coords[2])
    edge1 = p1.distance(p2)
    edge2 = p2.distance(p3)
    min_edge = min(edge1, edge2)
    if min_edge < 1.15:
        if cut_polygon(geom.geometry) == cut_num:
            return True
        return False
    return False


for i in range(2, 15):
    cut_num = i
    filter = gdf.apply(polygon_filter, axis=1)
    thin_gdf = gdf.loc[filter]
    num_cuts = np.array([cut_polygon(geom) for geom in thin_gdf['geometry']])
    avg_area = thin_gdf.area.sum() / sum(num_cuts)
    num_cuts_validate = np.round(np.array([area / avg_area for area in thin_gdf.area]))
    precision = abs((num_cuts_validate - num_cuts).sum()) / num_cuts.sum() * 100

    print(f'{cut_num}, {avg_area:.6f}, {avg_area * 0.95:.6f}, { precision:.4f}%')


2, 0.260064, 0.247061, 0.6449%
3, 0.285306, 0.271041, 0.1259%
4, 0.301217, 0.286157, 0.0362%
5, 0.310458, 0.294935, 0.1970%
6, 0.318833, 0.302891, 0.0202%
7, 0.325763, 0.309475, 0.0594%
8, 0.332177, 0.315568, 0.1428%
9, 0.336882, 0.320038, 0.1091%
10, 0.341462, 0.324389, 0.0000%
11, 0.346016, 0.328715, 0.0601%
12, 0.350439, 0.332917, 0.0695%
13, 0.353013, 0.335363, 0.0000%
14, 0.356044, 0.338242, 0.1020%


In [13]:
# # 2. 栅格转矢量，太慢了，用arcgis吧
# sp_ref = osr.SpatialReference()
# sp_ref.SetFromUserInput('EPSG:4326')

# source_band = src.read(1)
# mask_band_nodata = source_band.GetMaskBand()

# driver = ogr.GetDriverByName('ESRI Shapefile')
# dst_ds = driver.CreateDataSource(input_vector)
# dst_layer = dst_ds.CreateLayer('polygonized', srs=sp_ref)

# fd = ogr.FieldDefn('DN', ogr.OFTInteger)
# dst_layer.CreateField(fd)
# dst_field = 0

# # 参数  输入栅格图像波段、掩码图像波段、矢量化后的矢量图层、需要将DN值写入矢量字段的索引、算法选项
# gdal.Polygonize(source_band, mask_band_nodata, dst_layer, dst_field, [])

# dst_ds = None